# Wrangle and Analyze Twitter data


## Table of Contents
- [Introduction](#intro)
- [Gathering](#gather)
- [Assessing](#assess)
- [Issues](#issues)
- [Cleaning](#clean)
- [Insights & Visualizations](#visual)


<a id='intro'></a>
## Introduction

<a id='gather'></a>
## Gathering

#### 1. Working with `twitter-archive-enhanced.csv`
At first, we will read the `twitter-archive-enhanced.csv` file that is readily available for our analysis in Udacity project section.
To do that, we will first have to import pandas.

In [967]:
# Import pandas
import pandas as pd

In [968]:
# Integrate the twitter-archive-enhanced.csv file
df_arch = pd.read_csv('twitter-archive-enhanced.csv')

In [970]:
# Check that we correctly imported the file
df_arch.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


#### 2. Working with `image_predictions.tsv`
This file is available via the link provided in the Udacity project section so I will have to download this one programmatically.
To do that, I will have to import a couple of features first

In [2]:
# import libraries: requests and os
import requests
import os

In [7]:
# get the url
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

In [8]:
# download the file
with open(url.split('/')[-1], mode = 'wb') as file:
    file.write(response.content)

In [972]:
# check the correct import of the file
df_img = pd.read_csv('image-predictions.tsv', sep='\t')
df_img.tail(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2074,892420643555336193,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False


#### 3. Working with Twitter API
**Important** : This part of the analysis requires access to Twitter developer's account. Since I have still not been granted access to the data, I just have to import the code from Udacity's Resource section.

In [ ]:
# Import Tweepy and JSON libraries
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = arch.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)


In [973]:
# Create an empty data frame with only necessary columns
df_rt = pd.DataFrame(columns = ['tweet_id','retweet_count','favorite_count'])
df_rt

,tweet_id,retweet_count,favorite_count


In [974]:
import json

with open('tweet-json.txt', 'r') as outfile:
    for line in outfile:
        data = json.loads(line)
        tweet_id = data['id']
        retweet_count = data['retweet_count']
        favorite_count = data['favorite_count']
        df_rt = df_rt.append({
                    'tweet_id': tweet_id,
                    'retweet_count': retweet_count,
                    'favorite_count': favorite_count}, ignore_index=True)

In [975]:
#check if the data are there
df_rt.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


<a id='assess'></a>
## Assessing
So far, we have gathered three data frames from 3 different sources:
* **df_arch** - via`twitter-archive-enhanced.csv` from the Udacity Resource section;
* **df_img** - via `image-predictions.tsv` from the URL provided in the project;
* **df_rt** - from the Twitter API

Now, it's time to assess the data what we have gathered.
The issues will be listed below.

#### 1. Working with _df_arch_

In [976]:
# Display the df_arch table
df_arch.sample(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1384,700796979434098688,NaN,NaN,2016-02-19 21:39:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Daniel. He's a neat pup. Exotic af. Cu...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/700796979...,7,10,Daniel,None,None,None,None
1323,706265994973601792,NaN,NaN,2016-03-05 23:51:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Kara. She's been trying to solve that ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/706265994...,11,10,Kara,None,None,None,None
840,767122157629476866,NaN,NaN,2016-08-20 22:12:29 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rupert. You betrayed him with bath tim...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/767122157...,13,10,Rupert,None,None,None,None


In [977]:
# get the info about the table
df_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

Let's check if data types of particular interest are correct.

In [978]:
# Check the data type for timestamp
type(df_arch['timestamp'][0])

str

In [979]:
# Check the data type for retweeted_status_timestamp
type(df_arch['retweeted_status_timestamp'][0])

float

In [980]:
# Check the data type for source
type(df_arch['source'][0])

str

In [981]:
# Check for duplicates
sum(df_arch.duplicated())

0

In [982]:
# get the names of all the dogs in the table
df_arch.name.value_counts()

None        745
a            55
Charlie      12
Lucy         11
Cooper       11
           ... 
Vinscent      1
Bruiser       1
Skittle       1
Chaz          1
Gert          1
Name: name, Length: 957, dtype: int64

In [983]:
df_arch.rating_denominator.unique()

array([ 10,   0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40,
       130, 110,  16, 120,   2])

In [984]:
# Check the unique members of the Dogtionary dog type columns
df_arch.doggo.unique(), df_arch.floofer.unique(), df_arch.pupper.unique(), df_arch.puppo.unique()

(array(['None', 'doggo'], dtype=object),
 array(['None', 'floofer'], dtype=object),
 array(['None', 'pupper'], dtype=object),
 array(['None', 'puppo'], dtype=object))

#### 2. Working with _df_img_

In [985]:
# Display the df_img table
df_img.sample(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1970,868880397819494401,https://pbs.twimg.com/media/DA7iHL5U0AA1OQo.jpg,1,laptop,0.153718,False,French_bulldog,0.099984,True,printer,0.077130,False
1008,709207347839836162,https://pbs.twimg.com/media/CdecUSzUIAAHCvg.jpg,1,Chihuahua,0.948323,True,Italian_greyhound,0.017730,True,quilt,0.016688,False
1697,816697700272001025,https://pbs.twimg.com/media/C1V-K63UAAEUHqw.jpg,1,Chihuahua,0.756992,True,Pomeranian,0.052850,True,Maltese_dog,0.047608,True


In [986]:
# get the information about the table
df_img.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [987]:
# Check for duplicated rows
sum(df_img.duplicated())

0

#### 3. Working with _df_rt_

In [988]:
# Display the last rows of the df_rt table
df_rt.tail(3)

,tweet_id,retweet_count,favorite_count
2351,666033412701032449,47,128
2352,666029285002620928,48,132
2353,666020888022790149,532,2535


In [989]:
# get the information about the table
df_rt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
tweet_id          2354 non-null object
retweet_count     2354 non-null object
favorite_count    2354 non-null object
dtypes: object(3)
memory usage: 55.3+ KB


In [990]:
# Dig further for the data types
type(df_rt['tweet_id'][0]), type(df_rt['retweet_count'][0]), type(df_rt['favorite_count'][0])

(int, int, int)

<a id='issues'></a>
## Issues

### Quality
`df_arch` table:
* Some tweets are not original tweets, but replies or retweets;
* Some tweets are not about dogs;
* Multiple dog names in the column _name_ are incorrect;
* Rating denominators have multiple values that are different from 10;
* Erroneous data types for: `timestamp`, `in_reply_to_status_id`,`in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`;



`df_img` table:
* 2075 tweets covered instead of the full set of 2356 tweets in the archive;
* Different name cases in `p1`, `p2`, and `p3`;
* Non-existent dog breed types in columns: bookshop, syringe, flamingo...

`df_rt` table:
* Data types for `tweet_id`, `retweet_count`, `favorite_count` are not `int64`;
* Data for 3 tweets are missing

### Tidiness
`df_arch` table:
* Dogtionary dog type is split into 4 different columns (`doggo`, `floofer`, `pupper`, `puppo`);
* The column source is not tidy and can be adjusted into different source types

`df_img` table:
* AI guesses and probabilities are split into 3 different sections (`p1`, `p2`, `p3`)

Overall tidiness:
* After cleaning the three data frames, they can be merged into one - using `df_arch` as a main element.

<a id='clean'></a>
## Cleaning

We've assessed the quality and tidiness of our data sources, and now it's time to clean them one by one.
First, we need to make copies of our data frames.

In [992]:
# Create copies of the data frames
arch = df_arch.copy()
img = df_img.copy()
rt = df_rt.copy()


#### Define
* Some tweets are not original tweets, but replies or retweets

I need to slice these tweets and delete them from the data frame.

#### Code

In [993]:
# Drop the rows with non-null column in_reply_to_status_id
arch = arch[arch.in_reply_to_status_id.isnull()]

In [994]:
# Drop the rows with non-null column retweeted_status_id
arch = arch[arch.retweeted_status_id.isnull()]

#### Test

In [995]:
# Check if there are any non-null values for columns in_reply_to_status_id & in_reply_to_user_id
# Check if there are any non-null values for retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp
len(arch[arch.in_reply_to_status_id.notna()]), len(arch[arch.in_reply_to_user_id.notna()]), len(arch[arch.retweeted_status_id.notna()]), len(arch[arch.retweeted_status_user_id.notna()]), len(arch[arch.retweeted_status_timestamp.notna()]) 

(0, 0, 0, 0, 0)

In [996]:
# Drop the above columns that we do not need anymore
arch = arch.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis = 1)

In [998]:
arch.sample(1)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
401,824663926340194305,2017-01-26 17:02:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Wilson. Named after the volleyball. He...,https://twitter.com/dog_rates/status/824663926...,13,10,Wilson,None,None,None,None


#### Define
* Some tweets are not about dogs

Usually, these tweets are marked with the phrase "We only rate dogs", "Only send dogs", or some derived phrases. I will find these tweets and will delete them from the data frame.

#### Code

In [1018]:
# Using regular expressions, find tweets with the phrase "We only rate dogs" or "Only send (in) dogs", or "Not a dog" and get names
arch[arch.text.str.contains('(([Oo]nly|[Dd]o\s?n[o\']t)\.?\s+(.+\s+)?([Ss]end|[Rr]ate)\.?\s+(.+\s+)?([Dd]ogs))')].name.unique()

array(['None', 'quite', 'incredibly', 'a', 'very', 'an', 'getting',
       'unacceptable', 'infuriating', 'the', 'actually', 'Sammy'],
      dtype=object)

In [1017]:
arch[arch.text.str.contains('([Nn]ot\s+(a)\s+(dog))')]

array(["This is an Iraqi Speed Kangaroo. It is not a dog. Please only send in dogs. I'm very angry with all of you ...9/10 https://t.co/5qpBTTpgUt"],
      dtype=object)

As we can see, it seems that we are not missing any valid dog names, except Sammy - we can safely delete these rows from the data frame.

In [1056]:
len(arch)

2097

In [1057]:
# Save the Sammy the Snowball's tweet
sammy = arch[arch.tweet_id == 680055455951884288]
sammy

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1726,680055455951884288,2015-12-24 16:00:30 +0000,"<a href=""https://about.twitter.com/products/tw...","Meet Sammy. At first I was like ""that's a snow...",https://twitter.com/dog_rates/status/680055455...,10,10,Sammy,None,None,None,None


In [1058]:
# Get rid of non-dog tweets
arch = arch[~(arch.text.str.contains('(([Oo]nly)\.?\s+(.+\s+)?([Ss]end|[Rr]ate)\.?\s+(.+\s+)?([Dd]ogs))'))]
arch = arch[~(arch.text.str.contains('([Nn]ot\s+(a)\s+(dog))'))]

In [1060]:
# Append the Sammy the Snowball's tweet
arch = arch.append(sammy)

In [1062]:
# Check that it is there and only once
arch1[arch1.tweet_id == 680055455951884288]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1726,680055455951884288,2015-12-24 16:00:30 +0000,"<a href=""https://about.twitter.com/products/tw...","Meet Sammy. At first I was like ""that's a snow...",https://twitter.com/dog_rates/status/680055455...,10,10,Sammy,None,None,None,None


#### Test

In [1063]:
# Test if we cleaned it correctly
len(arch[arch.text.str.contains('(([Oo]nly)\.?\s+(.+\s+)?([Ss]end|[Rr]ate)\.?\s+(.+\s+)?([Dd]ogs))')]), len(arch[arch.text.str.contains('([Nn]ot\s+(a)\s+(dog))')])
# 1 is ok since we have our Sammy tweet

(1, 0)

#### Define
* Multiple dog names in the column _name_ are incorrect: **'a'** instead of a name, NaNs and None

Most of the tweets begin with "This is + name, but some a "This is + breed"
I will find them and extract names using regular expressions.
Also, some twetts start with "That is" - with wrong names as well.

#### Code

In [1064]:
arch[(arch.name.str.contains('([A-Z][a-z]*)') == False)].name.unique()

array(['such', 'a', 'not', 'one', 'my', 'his', 'actually', 'just', 'mad',
       'this', 'all', 'old', 'the', 'by', 'officially', 'life', 'light',
       'space', 'an', 'quite'], dtype=object)

In [1065]:
# Find all names that are beginning with a lowercase word/character and where there is a word "named" or "name is"
# it's highly likely that we could find a dog name there

arch[(arch.name.str.contains('([A-Z][a-z]*)') == False) & (arch.text.str.contains('(name[d]?\s?i?s?\s[A-Z]+[a-z]*)') == True)].text.unique()

array(["This is my dog. Her name is Zoey. She knows I've been rating other dogs. She's not happy. 13/10 no bias at all https://t.co/ep1NkYoiwB",
       "This is a Sizzlin Menorah spaniel from Brooklyn named Wylie. Lovable eyes. Chiller as hell. 10/10 and I'm out.. poof https://t.co/7E0AiJXPmI",
       'This is a Lofted Aphrodisiac Terrier named Kip. Big fan of bed n breakfasts. Fits perfectly. 10/10 would pet firmly https://t.co/gKlLpNzIl3',
       'This is a Tuscaloosa Alcatraz named Jacob (Yacōb). Loves to sit in swing. Stellar tongue. 11/10 look at his feet https://t.co/2IslQ8ZSc7',
       "This is a Helvetica Listerine named Rufus. This time Rufus will be ready for the UPS guy. He'll never expect it 9/10 https://t.co/34OhVhMkVr",
       'This is a Deciduous Trimester mix named Spork. Only 1 ear works. No seat belt. Incredibly reckless. 9/10 still cute https://t.co/CtuJoLHiDo',
       "This is a Rich Mahogany Seltzer named Cherokee. Just got destroyed by a snowball. Isn't very happy

In [1066]:
# Extract the necessary part of the string and replace the part preceding the actual name with nothing
arch.loc[arch.text.str.contains('name[d]?\s?i?s?\s[A-Z]+[a-z]*'), 'name'] = arch.text.str.extract('(name[d]?\s?i?s?\s[A-Z]+[a-z]*)', expand = False).str.replace('(name[d]?\s?i?s?\s)',"")

#### Test

In [1067]:
# Check
arch[arch.text.str.contains('name[d]?\s?i?s?\s[A-Z]+[a-z]*')]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
35,885518971528720385,2017-07-13 15:19:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I have a new hero and his name is Howard. 14/1...,https://twitter.com/4bonds2carbon/status/88551...,14,10,Howard,None,None,None,None
168,859607811541651456,2017-05-03 03:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Sorry for the lack of posts today. I came home...,https://twitter.com/dog_rates/status/859607811...,13,10,Zoey,None,None,None,puppo
852,765395769549590528,2016-08-16 03:52:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is my dog. Her name is Zoey. She knows I'...,https://twitter.com/dog_rates/status/765395769...,13,10,Zoey,None,None,None,None
1678,682047327939461121,2015-12-30 03:55:29 +0000,"<a href=""http://twitter.com/download/iphone"" r...",We normally don't rate bears but this one seem...,https://twitter.com/dog_rates/status/682047327...,10,10,Thea,None,None,None,None
1734,679736210798047232,2015-12-23 18:51:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This pup's name is Sabertooth (parents must be...,https://twitter.com/dog_rates/status/679736210...,9,10,Sabertooth,None,None,None,None
1853,675706639471788032,2015-12-12 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Sizzlin Menorah spaniel from Brookly...,https://twitter.com/dog_rates/status/675706639...,10,10,Wylie,None,None,None,None
1955,673636718965334016,2015-12-06 22:54:44 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Lofted Aphrodisiac Terrier named Kip...,https://twitter.com/dog_rates/status/673636718...,10,10,Kip,None,None,None,None
2034,671743150407421952,2015-12-01 17:30:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Tuscaloosa Alcatraz named Jacob (Yac...,https://twitter.com/dog_rates/status/671743150...,11,10,Jacob,None,None,None,None
2066,671147085991960577,2015-11-30 02:01:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Helvetica Listerine named Rufus. Thi...,https://twitter.com/dog_rates/status/671147085...,9,10,Rufus,None,None,None,None
2116,670427002554466305,2015-11-28 02:20:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Deciduous Trimester mix named Spork....,https://twitter.com/dog_rates/status/670427002...,9,10,Spork,None,None,None,None


In [1068]:
arch[(arch.name.str.contains('([A-Z][a-z]*)') == False)].text.unique()

array(["I've yet to rate a Venezuelan Hover Wiener. This is such an honor. 14/10 paw-inspiring af (IG: roxy.thedoxy) https://t.co/20VrLAA8ba",
       'Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \n(IG: puffie_the_chow) https://t.co/ghXBIIeQZF',
       "There's going to be a dog terminal at JFK Airport. This is not a drill. 10/10  \nhttps://t.co/dp5h9bCwU7",
       "Occasionally, we're sent fantastic stories. This is one of them. 14/10 for Grace https://t.co/bZ4axuH6OK",
       'Here is a perfect example of someone who has their priorities in order. 13/10 for both owner and Forrest https://t.co/LRyMrU7Wfq',
       "This is one of the most inspirational stories I've ever come across. I have no words. 14/10 for both doggo and owner https://t.co/I5ld3eKD5k",
       'What jokester sent in a pic without a dog in it? This is not @rock_rates. This is @dog_rates. Thank you ...10/10 https://t.co/nDPaYHrtNX',
       'That is Quizno. 

#### Code (2)
Several of these dogs have their names deeply hidden in the tweets, so I will extract them manually.

In [1081]:
# Let's take care of the little ones
arch.loc[arch.text.str.contains('(baby Rand Paul)'), 'name'] = 'Rand Paul'
arch.loc[arch.text.str.contains('(puffie_the_chow)'), 'name'] = 'Puffie'
arch.loc[arch.text.str.contains('(14/10 for Grace)'), 'name'] = 'Grace'
arch.loc[arch.text.str.contains('(and Forrest)'), 'name'] = 'Forrest'
arch.loc[arch.text.str.contains('(That is Quizno)'), 'name'] = 'Quizno'
arch.loc[arch.text.str.contains('(Butternut Cumberfloof)'), 'name'] = 'Butternut Cumberfloof'
arch.loc[arch.text.str.contains('(Toblerone)'), 'name'] = 'Toblerone'
arch.loc[arch.text.str.contains('(Vesuvius)'), 'name'] = 'Vesuvius'
arch.loc[arch.text.str.contains('(Parthenon)'), 'name'] = 'Parthenon'
arch.loc[arch.text.str.contains('(Piers Morgan)'), 'name'] = 'Piers Morgan'

#### Test (2)

In [1082]:
arch[(arch.name.str.contains('([A-Z][a-z]*)') == False)].text.unique()

array(["I've yet to rate a Venezuelan Hover Wiener. This is such an honor. 14/10 paw-inspiring af (IG: roxy.thedoxy) https://t.co/20VrLAA8ba",
       "There's going to be a dog terminal at JFK Airport. This is not a drill. 10/10  \nhttps://t.co/dp5h9bCwU7",
       "This is one of the most inspirational stories I've ever come across. I have no words. 14/10 for both doggo and owner https://t.co/I5ld3eKD5k",
       'What jokester sent in a pic without a dog in it? This is not @rock_rates. This is @dog_rates. Thank you ...10/10 https://t.co/nDPaYHrtNX',
       "This is one of the most reckless puppers I've ever seen. How she got a license in the first place is beyond me. 6/10 https://t.co/z5bAdtn9kd",
       'This is a mighty rare blue-tailed hammer sherk. Human almost lost a limb trying to take these. Be careful guys. 8/10 https://t.co/TGenMeXreW',
       'Viewer discretion is advised. This is a terrible attack in progress. Not even in water (tragic af). 4/10 bad sherk https://t.co/L3U0j1

In [1084]:
# Name the other dogs "Unknown"
arch.loc[arch.name.str.contains('([A-Z][a-z]*)') == False, 'name'] = 'Unknown'
# And also the dog names described as 'None'
arch.loc[arch.name == 'None', 'name'] = 'Unknown'

In [1085]:
# Do the check
arch.loc[arch.name.str.contains('([A-Z][a-z]*)') == False, 'name']

Series([], Name: name, dtype: object)

#### Define
* Rating denominators have multiple values that are different from 10

I need to check how many values are not equal to 10, and what is the logiс behind the calculation before acting on it.

#### Code
Let's find all the denominators out there that are not equal to 10, but in the text there is something like "/10", so the correct rating is not taken into account.

In [1088]:
# Query the above assumption (using text)
arch[(arch.rating_denominator != 10) & (arch.text.str.contains('(\/10)'))].text.unique()

array(['After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ',
       'Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a',
       'This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq',
       'This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5',
       'This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv'],
      dtype=object)

In [1089]:
# Extract the correct rating for these tweets
arch.loc[(arch.rating_denominator != 10) & (arch.text.str.contains('(\/10)')), 'rating_denominator'] = 10
arch.loc[(arch.rating_denominator != 10) & (arch.text.str.contains('(\/10)')), 'rating_numerator'] = arch.text.str.extract('(\d+\/10)', expand = False).str.replace('(\d+)',"")

#### Test

In [1091]:
arch[arch.rating_denominator != 10].text.unique()

array(['The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd',
       'Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx',
       'Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE',
       'Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv',
       "Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1",
       "Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12",
       'From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK',
       "Here is a whole flock of puppers.  60/50 I'll take the lot https://t.co/9dpcw6MdWa",
       "Happy We

Clearly, there is one tweet without a proper rating (# 810984652412424192 - featuring Sam) - we would need to give this guy a 10/10 (for the consistency of our data frame)
#### Code (2)

In [1092]:
# Rate Sam as 10/10
arch.loc[arch.tweet_id == 810984652412424192, 'rating_numerator'] = 10
arch.loc[arch.tweet_id == 810984652412424192, 'rating_denominator'] = 10

Now, we will proportionally decrease ratings with higher denominators to arrive to the common denominator of 10.

In [1093]:
# Change the ratings proportionally while keeping values as integers
arch.loc[arch.rating_denominator != 10, 'rating_numerator'] = (arch.loc[arch.rating_denominator != 10, 'rating_numerator'] / arch.loc[arch.rating_denominator != 10, 'rating_denominator'] * 10).astype(int)
arch.loc[arch.rating_denominator != 10, 'rating_denominator'] = (arch.loc[arch.rating_denominator != 10, 'rating_denominator'] / arch.loc[arch.rating_denominator != 10, 'rating_denominator'] * 10).astype(int)

#### Test (2)

In [1094]:
# Test if there are any denominators not equal to 10
len(arch[arch.rating_denominator != 10])

0

In [1095]:
# Test one tweet for consistency
arch[arch.tweet_id == 677716515794329600]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1779,677716515794329600,2015-12-18 05:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",IT'S PUPPERGEDDON. Total of 144/120 ...I think...,https://twitter.com/dog_rates/status/677716515...,12,10,Unknown,None,None,None,None


#### Define
* Erroneous data types for: `timestamp`, `in_reply_to_status_id`,`in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`

I only need to convert the `timestamp` column data type since the other ones are gone.

#### Code
Let's convert the column data type

In [1096]:
# Convert 'timestamp' to datetime
arch['timestamp'] = pd.to_datetime(arch['timestamp'])

#### Test

In [1098]:
arch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2031 entries, 0 to 1726
Data columns (total 12 columns):
tweet_id              2031 non-null int64
timestamp             2031 non-null datetime64[ns, UTC]
source                2031 non-null object
text                  2031 non-null object
expanded_urls         2028 non-null object
rating_numerator      2031 non-null int64
rating_denominator    2031 non-null int64
name                  2031 non-null object
doggo                 2031 non-null object
floofer               2031 non-null object
pupper                2031 non-null object
puppo                 2031 non-null object
dtypes: datetime64[ns, UTC](1), int64(3), object(8)
memory usage: 286.3+ KB


#### Define
* Dogtionary dog type is split into 4 different columns (doggo, floofer, pupper, puppo)

Need to use the for loop to go through the Dogtionary columns to create a single Dogtionary column.
The `pd.melt()` is not available because some of the rows have four 'None' values consecutively.
#### Code

In [1099]:
# Create a new column and fill it in with a simple string
arch['dogtionary_type'] = "a"
arch.head(1)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,dogtionary_type
0,892420643555336193,2017-08-01 16:23:56+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,a


In [1100]:
# Apply the for loop
for i in range(len(arch)):
    if arch.iloc[i].doggo == 'doggo':
        arch.dogtionary_type.iloc[i] = 'doggo'
    elif arch.iloc[i].floofer == 'floofer':
        arch.dogtionary_type.iloc[i] = 'floofer'
    elif arch.iloc[i].pupper == 'pupper':
        arch.dogtionary_type.iloc[i] = 'pupper'
    elif arch.iloc[i].puppo == 'puppo':
        arch.dogtionary_type.iloc[i] = 'puppo'
    else: arch.dogtionary_type.iloc[i] = 'unknown'

/Users/nicklemonparty/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [1102]:
# drop the 4 columns that we do not need anymore
arch = arch.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis = 1)

#### Test

In [1103]:
# Get a sample of 10 lines of the data frame
arch.sample(10)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dogtionary_type
2190,668960084974809088,2015-11-24 01:11:27+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jaycob. He got scared of the vacuum. Hide...,https://twitter.com/dog_rates/status/668960084...,10,10,Jaycob,unknown
459,817827839487737858,2017-01-07 20:18:46+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Buddy. He ran into a glass door once. ...,https://twitter.com/dog_rates/status/817827839...,13,10,Buddy,unknown
1672,682389078323662849,2015-12-31 02:33:29+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Meet Brody. He's a Downton Abbey Falsetto. Add...,https://twitter.com/dog_rates/status/682389078...,9,10,Brody,unknown
237,847157206088847362,2017-03-29 18:43:12+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Meet Daisy. She's been pup for adoption for mo...,"https://www.petfinder.com/petdetail/37334596,h...",11,10,Daisy,unknown
2330,666362758909284353,2015-11-16 21:10:36+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Unique dog here. Very small. Lives in containe...,https://twitter.com/dog_rates/status/666362758...,6,10,Unknown,unknown
1213,715220193576927233,2016-03-30 16:52:36+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Nico. His selfie game is strong af. Ex...,https://twitter.com/dog_rates/status/715220193...,10,10,Nico,unknown
107,871762521631449091,2017-06-05 16:15:56+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Rover. As part of pupper protocol he h...,https://twitter.com/dog_rates/status/871762521...,12,10,Rover,pupper
376,828046555563323392,2017-02-05 01:04:17+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Sailer. He waits on the roof for his o...,https://twitter.com/dog_rates/status/828046555...,13,10,Sailer,unknown
903,758405701903519748,2016-07-27 20:56:24+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Odie. He falls asleep wherever he want...,https://twitter.com/dog_rates/status/758405701...,10,10,Odie,unknown
626,795076730285391872,2016-11-06 01:33:58+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Bailey. She loves going down slides bu...,https://twitter.com/dog_rates/status/795076730...,11,10,Bailey,unknown


#### Define

* The column source is not tidy and can be adjusted into different source types

#### Code

In [1104]:
# Check the unique names of the source column
arch.source.unique()

array(['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'],
      dtype=object)

In [1105]:
# Modify the source column using RegEx and replace function
arch['source'] = arch['source'].str.extract('(>.+<)')
arch['source'] = arch['source'].str.replace(">","")
arch['source'] = arch['source'].str.replace("<","")

In [1106]:
# Check the names
arch.source.unique()

array(['Twitter for iPhone', 'Twitter Web Client', 'Vine - Make a Scene',
       'TweetDeck'], dtype=object)

In [1107]:
# Rename columns
arch.loc[arch.source == "Vine - Make a Scene", 'source'] = 'Vine'
arch.loc[arch.source == "Twitter for iPhone", 'source'] = 'iPhone'
arch.loc[arch.source == "Twitter Web Client", 'source'] = 'Web'

#### Test

In [1108]:
# Check the names
arch.source.unique()

array(['iPhone', 'Web', 'Vine', 'TweetDeck'], dtype=object)

#### Define
* 2075 tweets covered instead of the full set of 2356 tweets in the archive;

Here, we cannot really do anything because the file does not have any extension or another version, so we will just have to accept that some tweets are missing.

Let's move on to the `img` data frame.

In [1109]:
img.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


#### Define
* AI guesses and probabilities are split into 3 different sections (p1, p2, p3)
* Non-existent dog breed types in columns: bookshop, syringe, flamingo...

I see that some predictions of `p1`, `p2`, and `p3` are false. Therefore, I will create a column `breed` and use:
 * the `p1` dog name if `p1_dog` is True;
 * `p2` is `p1_dog` is False but `p2_dog` is True; 
 * `p3` if both `p1` and `p2` are False, but `p3` is True;
 * Finally, I will name the breed 'unknown' if none of the AI's predictions found a dog on the photo

#### Code

In [1110]:
# Create a new column and fill it in with a simple string
img['dog_breed'] = "a"
img.head(1)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,dog_breed
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True,a


In [1111]:
# Apply the for loop
for i in range(len(img)):
    if img.iloc[i].p1_dog == True:
        img.dog_breed.iloc[i] = img.iloc[i].p1
    elif img.iloc[i].p2_dog == True:
        img.dog_breed.iloc[i] = img.iloc[i].p2
    elif img.iloc[i].p3_dog == True:
        img.dog_breed.iloc[i] = img.iloc[i].p3
    else: img.dog_breed.iloc[i] = 'unknown'

In [1112]:
img.columns

Index(['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'dog_breed'],
      dtype='object')

In [1113]:
# drop the 8 columns that we do not need anymore
img = img.drop(['img_num', 'p1', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], axis = 1)

In [1114]:
# Rename the column "p1_conf" to "breed_conf"
img = img.rename(columns={'p1_conf':'breed_conf'})

In [1117]:
# Rearrange columns
img = img[['tweet_id', 'dog_breed', 'breed_conf', 'jpg_url']]

#### Test

In [1118]:
# Test if the above code worked
img.head(1)

,tweet_id,dog_breed,breed_conf,jpg_url
0,666020888022790149,Welsh_springer_spaniel,0.465074,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg


#### Define
* Different namecases in `p1`, `p2`, and `p3`;

The problem became much easier since all of these columns were dropped, and now we only had `dog_breed` to adjust. I use lower case on every breed in this column.
#### Code

In [1119]:
img['dog_breed'] = img['dog_breed'].str.lower()

#### Test

In [1120]:
# Check if chahges were applied
img.sample(10)

,tweet_id,dog_breed,breed_conf,jpg_url
922,702217446468493312,golden_retriever,0.242419,https://pbs.twimg.com/media/Cb7HCMkWEAAV9zY.jpg
1117,725786712245440512,chow,0.335761,https://pbs.twimg.com/media/ChKDKmIWIAIJP_e.jpg
335,672205392827572224,pug,0.952613,https://pbs.twimg.com/media/CVQnPMrVAAAzShR.jpg
2058,888917238123831296,golden_retriever,0.714719,https://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg
1226,745057283344719872,shetland_sheepdog,0.963985,https://pbs.twimg.com/media/Clb5pLJWMAE-QS1.jpg
749,687818504314159109,lakeland_terrier,0.873029,https://pbs.twimg.com/media/CYufR8_WQAAWCqo.jpg
1976,870308999962521604,greater_swiss_mountain_dog,0.622752,https://pbs.twimg.com/media/DBP1asiUAAEKZI5.jpg
48,666826780179869698,maltese_dog,0.359383,https://pbs.twimg.com/media/CUELa0NUkAAscGC.jpg
837,694329668942569472,boxer,0.990060,https://pbs.twimg.com/media/CaLBJmOWYAQt44t.jpg
771,689517482558820352,pembroke,0.799319,https://pbs.twimg.com/media/CZGofjJW0AINjN9.jpg


Now, let's move on the the last data frame: `rt`

#### Define
* Data types for `tweet_id`, `retweet_count`, `favorite_count` are not `int64`

Need to change the data type programatically using `astype(int)`
#### Code

In [1121]:
# Change the data type
rt = rt[['tweet_id','retweet_count','favorite_count']].astype(int)

#### Test

In [1122]:
# Check the information about the data frame
rt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
tweet_id          2354 non-null int64
retweet_count     2354 non-null int64
favorite_count    2354 non-null int64
dtypes: int64(3)
memory usage: 55.3 KB


#### Define 
* After cleaning the three data frames, they can be merged into one - using `df_arch` as a main element.

Now we can merge all the three tables by `tweet_id`
#### Code

In [1123]:
# Merge arch and img data frames
arch_img = pd.merge(arch,img,how='left',on='tweet_id')

In [1124]:
# Merge the resulting data frame with rt to get the full table
full = pd.merge(arch_img,rt,how='left',on='tweet_id')

In [1125]:
# Get the comprehensive total column list
full.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'dogtionary_type',
       'dog_breed', 'breed_conf', 'jpg_url', 'retweet_count',
       'favorite_count'],
      dtype='object')

In [1126]:
# Rearrange columns
full = full[['tweet_id', 'text', 'retweet_count', 'favorite_count', 'dog_breed', 'breed_conf', 
             'name', 'dogtionary_type', 'rating_numerator', 'rating_denominator', 'source',
             'timestamp','expanded_urls', 'jpg_url']]

#### Test

In [1127]:
# Get the first line of the full data frame
full.head(1)

,tweet_id,text,retweet_count,favorite_count,dog_breed,breed_conf,name,dogtionary_type,rating_numerator,rating_denominator,source,timestamp,expanded_urls,jpg_url
0,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,8853,39467,unknown,0.097049,Phineas,unknown,13,10,iPhone,2017-08-01 16:23:56+00:00,https://twitter.com/dog_rates/status/892420643...,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg


In [1128]:
#Get the detailed information about the data frame
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2031 entries, 0 to 2030
Data columns (total 14 columns):
tweet_id              2031 non-null int64
text                  2031 non-null object
retweet_count         2031 non-null int64
favorite_count        2031 non-null int64
dog_breed             1906 non-null object
breed_conf            1906 non-null float64
name                  2031 non-null object
dogtionary_type       2031 non-null object
rating_numerator      2031 non-null int64
rating_denominator    2031 non-null int64
source                2031 non-null object
timestamp             2031 non-null datetime64[ns, UTC]
expanded_urls         2028 non-null object
jpg_url               1906 non-null object
dtypes: datetime64[ns, UTC](1), float64(1), int64(5), object(7)
memory usage: 238.0+ KB


In [1133]:
# Fill null values in dog_breed with "unknown" and breed_conf with 0 (as we are sure these dogs have some breed)
full['dog_breed'] = full['dog_breed'].fillna('unknown')
full['breed_conf'] = full['breed_conf'].fillna(0)
# Fill null values in expanded_urls and jpg_url with 'not available'
full['expanded_urls'] = full['expanded_urls'].fillna('not available')
full['jpg_url'] = full['jpg_url'].fillna('not available')

In [1134]:
# Check the modifications
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2031 entries, 0 to 2030
Data columns (total 14 columns):
tweet_id              2031 non-null int64
text                  2031 non-null object
retweet_count         2031 non-null int64
favorite_count        2031 non-null int64
dog_breed             2031 non-null object
breed_conf            2031 non-null float64
name                  2031 non-null object
dogtionary_type       2031 non-null object
rating_numerator      2031 non-null int64
rating_denominator    2031 non-null int64
source                2031 non-null object
timestamp             2031 non-null datetime64[ns, UTC]
expanded_urls         2031 non-null object
jpg_url               2031 non-null object
dtypes: datetime64[ns, UTC](1), float64(1), int64(5), object(7)
memory usage: 238.0+ KB


In [1135]:
# Save the full dataframe to a CSV file
full.to_csv('twitter_full_database.csv', index = False)